In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import os
import copy
from typing import Union, List, Dict, Any, cast

In [ ]:
from torch.hub import load_state_dict_from_url

class VGG(nn.Module):

    def __init__(
        self,
        features: nn.Module,
        num_classes: int = 1000,
        init_weights: bool = True,
        eps: float = 0.3,
        bin_size : int = 6
    ) -> None:
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        
        self.bin_size = bin_size
        self.eps = eps
        self.max_record = torch.full([512 * 7 * 7], float('-inf'))
        self.min_record = torch.full([512 * 7 * 7], float('inf'))

        self.neuron_covered = torch.full([512 * 7 * 7], True)
        self.bin_covered = torch.full([512 * 7 * 7, self.bin_size], True)
        self.upper_covered = torch.full([512 * 7 * 7], True)
        self.lower_covered = torch.full([512 * 7 * 7], True)
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x: torch.Tensor, isTrain: bool) -> torch.Tensor:
        x = self.features(x)
        x = x.view(-1, 512 * 7 * 7)

        if isTrain:
            self.max_record = torch.max(self.max_record, torch.max(x, 0).values)
            self.min_record = torch.min(self.max_record, torch.min(x, 0).values)
        else:
            self.neuron_covered = self.neuron_covered.logical_and(torch.min(x >= self.eps, 0).values)
            self.upper_covered = self.upper_covered.logical_and(torch.min(x >= self.max_record, 0).values)
            self.lower_covered = self.lower_covered.logical_and(torch.min(x <= self.min_record, 0).values)
            
            min_bound = self.min_record
            diff = (self.max_record - self.min_record) / self.bin_size
            max_bound = self.min_record + diff
            for i in range(self.bin_size):
                self.bin_covered[:, i] = self.bin_covered[:, i].logical_and(torch.min(x >= min_bound, 0).values).logical_and(torch.min(x <= max_bound, 0).values)
                min_bound = min_bound + diff
                max_bound = max_bound + diff
        x = x.view(-1, 512, 7, 7)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self) -> None:
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg: List[Union[str, int]], batch_norm: bool = False) -> nn.Sequential:
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

def get_vgg(arch: str, cfg: str, **kwargs: Any) -> VGG:
    kwargs['init_weights'] = False
    model = VGG(make_layers(torchvision.models.vgg.cfgs[cfg]), **kwargs)
    state_dict = load_state_dict_from_url(torchvision.models.vgg.model_urls[arch], progress=True)
    model.load_state_dict(state_dict)
    return model

In [ ]:
model = get_vgg("vgg11", 'A')
images = torch.rand([128, 3, 224, 224])
tests = torch.rand([16, 3, 224, 224])
model(images, True)
model(tests, False)
print(model.neuron_covered)
print(model.upper_covered)
print(model.lower_covered)
print(model.bin_covered)

In [ ]:
print(model.neuron_covered.count_nonzero())
print(model.upper_covered.count_nonzero())
print(model.lower_covered.count_nonzero())
print(model.bin_covered.count_nonzero())
print(model.neuron_covered.shape)
print(model.upper_covered.shape)
print(model.lower_covered.shape)
print(model.bin_covered.shape)

```
uname = input()
!git config --global user.email '$uname@gmail.com'
!git config --global user.name '$uname'

from getpass import getpass
password = getpass('Password:')
!git clone https://github.com/HyunjoonCho/cs453-project-21s
%cd myrepo
# create a file, then add it to stage
!git add tester.ipynb
!git commit -m 'commit message'  # commit in Colab
!git push origin master          # push to github
```